In [1]:
%cd /home/wzt/src/moco-v3-3d/
import vits_3d
import torch
import transforms as my_transforms
import datasets as my_datasets
import monai
import moco
import moco.builder
from functools import partial
import copy
from tqdm.auto import tqdm
import SimpleITK as sitk
import numpy as np
import pandas as pd
from pathlib import Path

x = torch.load("x.pth")
checkpoint = torch.load("/home/wzt/src/moco-v3-3d/checkpoint_0099.pth.tar", map_location="cpu")

/home/wzt/src/moco-v3-3d


In [ ]:
def loadstate(model, state_dict):
    state_dict = copy.deepcopy(state_dict)
    linear_keyword = 'head'

    # freeze all layers but the last fc
    for name, param in model.named_parameters():
        if name not in ['%s.weight' % linear_keyword, '%s.bias' % linear_keyword]:
            param.requires_grad = False
    # init the fc layer
    getattr(model, linear_keyword).weight.data.normal_(mean=0.0, std=0.01)
    getattr(model, linear_keyword).bias.data.zero_()

    # rename moco pre-trained keys
    for k in list(state_dict.keys()):
        # retain only base_encoder up to before the embedding layer
        if k.startswith('module.base_encoder') and not k.startswith('module.base_encoder.%s' % linear_keyword):
            # remove prefix
            state_dict[k[len("module.base_encoder."):]] = state_dict[k]
        # delete renamed or unused k
        del state_dict[k]

    msg = model.load_state_dict(state_dict, strict=False)
    assert set(msg.missing_keys) == {"%s.weight" % linear_keyword, "%s.bias" % linear_keyword}
    return model

In [ ]:
model = vits_3d.__dict__["vit_3d_conv_small"](img_size=48, in_chans=1)
model = loadstate(model, checkpoint['state_dict'])
model.eval()
model(x)

In [ ]:
model = moco.builder.MoCo_ViT(
    partial(vits_3d.__dict__["vit_3d_conv_small"], img_size=48, in_chans=1),
    256, 4096, 1.0).base_encoder
model = loadstate(model, checkpoint['state_dict'])

In [ ]:
for k, v in checkpoint['state_dict'].items():
    if k.startswith("module.base_encoder.patch_embed.proj.1."):
        print(k)
        print(v)

In [ ]:
nbn = torch.load("nan_bn3d.pth")
x = torch.randn(4, 48, 24, 24, 24)

In [ ]:
nbn.train()
nbn(x).sum()

In [ ]:
nbn.eval()
nbn.running_mean = torch.zeros_like(nbn.running_mean)
nbn.running_var = torch.ones_like(nbn.running_var)
nbn(x).sum()

In [ ]:
nbn.eval()
torch.isnan(nbn(torch.randn(4, 48, 24, 24, 24))).any()

In [ ]:
nbn.state_dict()

In [ ]:
bn.state_dict()

In [ ]:
torch.nn.BatchNorm3d(48).running_var

In [ ]:
bn = torch.nn.BatchNorm3d(48)
bn.weight = nbn.weight
bn.eval()

torch.isnan(bn(torch.randn(4, 48, 24, 24, 24))).any()

In [2]:
def make_transform(patch_size):
    aug_block = my_transforms.aug_transform(size=patch_size)
    return monai.transforms.Compose([
        monai.transforms.EnsureChannelFirst(channel_dim='no_channel'),
        my_transforms.Duplicate(transforms1=aug_block, transforms2=aug_block),
    ])
    
traindir = '/media/wzt/plum14t/wzt/ProcressedData/ROI_resize48/images_cut_48_resize/'
train_dataset = my_datasets.SSLNoCropDataset(traindir, size=48, transform=make_transform(48), enable_negatives=False)
print("train_dataset len: ", len(train_dataset))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=4, shuffle=False,
    num_workers=0, pin_memory=True)

train_dataset len:  39939


In [4]:
for images, _ in tqdm(train_dataset):
    x1 = images[0]
    print(type(x1))
    x1 = monai.transforms.ToTensor(track_meta=False)(x1)
    print(type(x1))
    break

  0%|          | 0/39939 [00:00<?, ?it/s]

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [ ]:
for img_path in tqdm(Path("/media/wzt/plum14t/wzt/ProcressedData/ROI_resize48/images_cut_48_resize/").glob("*.nii.gz")):
    name = img_path.name.lower()
    if "c0" in name or "c2" in name:
        sym_path = Path("/media/wzt/plum14t/wzt/ProcressedData/ROI_resize48/train/") / img_path.name
        sym_path.symlink_to(img_path)

In [ ]:
data = []
i = 0
for images, _ in tqdm(train_dataset):
    row = train_dataset.get_row(i)
    try:
        row.update(dict(min1=images[0].min().item(), max1=images[0].max().item(), min2=images[1].min().item(), max2=images[1].max().item(), mean1=images[0].mean().item(), mean2=images[1].mean().item(), std1=images[0].std().item(), std2=images[1].std().item()))
    except Exception as e:
        print(i, e)
    data.append(row)
    i += 1
pd.DataFrame(data).to_csv("data3.csv", index=False)

In [ ]:
model = moco.builder.MoCo_ViT(
    partial(vits_3d.__dict__["vit_3d_conv_small"], img_size=48, in_chans=1),
    256, 4096, 1.0).cuda()

optimizer = torch.optim.AdamW(model.parameters(), 0.6, weight_decay=1e-6)

In [ ]:
def make_transform(patch_size):
    aug_block = my_transforms.aug_transform(size=patch_size, prob=1)
    return monai.transforms.Compose([
        monai.transforms.EnsureChannelFirst(channel_dim='no_channel'),
        my_transforms.Duplicate(transforms1=aug_block, transforms2=aug_block),
    ])
    
traindir = '/mnt/tmp/nan/'
train_dataset = my_datasets.SSLNoCropDataset(traindir, size=48, transform=make_transform(48), enable_negatives=False)
print("train_dataset len: ", len(train_dataset))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=5, shuffle=False,
    num_workers=0, pin_memory=True)

for e in tqdm(range(1000)):
    # for images, _ in train_loader:
        x1 = images[0].cuda()
        x2 = images[1].cuda()

        loss = model(x1=torch.zeros_like(x1), x2=torch.zeros_like(x1), m=0.99)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
model.base_encoder.patch_embed.proj[1].state_dict()["running_mean"], model.base_encoder.patch_embed.proj[1].state_dict()["running_var"]